In [1]:
from data import dataset, PlantOrgansDataset
from preprocessing import preprocess_image_and_mask
import torchvision.transforms.v2 as T
import torch
import numpy as np
from alexnet import MyTransform, SlidingWindow, get_extractor, get_feature, get_model
from train import device, pixel_validate, patch_loss, patch_validate, evaluate, fit
import torch.utils.data as data_utils
from kmeans import KNN
from evaluate import calculate_metrics
from torch.utils.data import DataLoader
from alexnet_knn import perform_segmentation, save_features, segmentation_image, retrieve_features, read_features, load_and_merge_knn_features, get_feature_filename
import umap
from sklearn.manifold import trustworthiness
import pickle
import os
import lz4

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Using cache found in C:\Users\pc/.cache\torch\hub\pytorch_vision_v0.10.0
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in C:\Users\pc/.cache\torch\hub\pytorch_vision_v0.10.0


In [2]:
from transforms import image_to_tensor, mask_to_tensor

Constants

In [3]:
src_path = "C:\\Users\\pc\\Documents\\repos\\mp-2\\nn\\nn-lab2\\"

Init datasets

In [4]:
train_validation_data = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = train_validation_data['train']
validation_dataset = train_validation_data['test']
test_dataset = dataset['validation']


In [5]:
config = {
    "layer_name": "classifier.0",
    "sliding_window_size": 32, 
    "sliding_window_step": 11, 
    "feature_depth": 9216,
    "resize_size": 512 
}

In [6]:
model = get_model(device)

Using cache found in C:\Users\pc/.cache\torch\hub\pytorch_vision_v0.10.0
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
image_to_tensor_512 = image_to_tensor(config["resize_size"])

def mask_of_uniform_size(resize_size: int):
    return T.Compose([
    T.Resize((resize_size, resize_size), interpolation=T.InterpolationMode.NEAREST_EXACT),
    mask_to_tensor,
])

mask_to_tensor_512 = mask_of_uniform_size(config["resize_size"])

c:\ProgramData\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [131]:
train_size = 30
test_size = 10

In [132]:
train_dataset_images = torch.concat(list(map(lambda image: image_to_tensor_512(image).unsqueeze(0), train_dataset[0:train_size]['image'])))

In [133]:
train_dataset_masks = torch.concat(list(map(lambda mask: mask_to_tensor_512(mask), train_dataset[0:train_size]['label'])))

In [134]:
test_dataset_images = torch.concat(list(map(lambda image: image_to_tensor_512(image).unsqueeze(0), test_dataset[0:test_size]['image'])))
# test_dataset_masks = mask_to_tensor(test_dataset[0:test_size]['label'])

In [135]:
with open(os.path.join("umap_model_5.lz4"), 'rb') as f:
    reducer = pickle.load(f)

In [154]:
train_reduced_features = []

In [155]:
for i in range(train_size):
    file_name = get_feature_filename(i, src_path, config["layer_name"], config["sliding_window_size"], config["sliding_window_step"], "train", config["resize_size"])
    reduced_file_name = file_name  + "_reduced.lx4"
    torch.cuda.empty_cache()
    if os.path.isfile(reduced_file_name):
        _reduced_features = read_features(i, src_path, reduced_file_name).to("cpu")
    else:
        if os.path.isfile(file_name):
            _features = read_features(i, src_path, file_name).to("cpu")
        else:
            _features = retrieve_features(train_dataset_images[i].unsqueeze(0).to(device), config["layer_name"], config["sliding_window_size"], config["sliding_window_step"], config["feature_depth"], model).to("cpu")
            save_features(_features, file_name)
        features_transformed = reducer.transform(_features.view(_features.size(0) * _features.size(1), -1))
        _reduced_features = torch.tensor(features_transformed)
        save_features(_reduced_features, reduced_file_name)
    train_reduced_features.append(_reduced_features)
reduced_train_features_tensor = torch.concat(train_reduced_features)
torch.cuda.empty_cache()

In [138]:
from transforms import mask_of_uniform_size
def get_ground_truth_for_indices(indices, dataset):
    ground_truths = []
    for i in indices:
        mask_patches = SlidingWindow(32, 11)(dataset[i].unsqueeze(0).unsqueeze(0))
        mask_patches_ = mask_patches.view(mask_patches.size(0), mask_patches.size(1), -1)
        ground_truths.append(torch.mode(mask_patches_, dim=2).values.to(dtype=torch.int8).view(-1))
        del mask_patches, mask_patches_
    merged_ground_truths = torch.cat(ground_truths, dim=0).to(device)
    del ground_truths
    return merged_ground_truths

In [156]:
train_ground_truth = get_ground_truth_for_indices(range(0, train_size), train_dataset_masks)

In [157]:
knn = KNN(reduced_train_features_tensor.to(device), train_ground_truth.to(device), 5, save_memory=True)

In [158]:
test_reduced_features = []
for i in range(test_size):
    file_name = get_feature_filename(i, src_path, config["layer_name"], config["sliding_window_size"], config["sliding_window_step"], "test", config["resize_size"])
    reduced_file_name = file_name  + "_reduced.lx4"
    torch.cuda.empty_cache()
    if os.path.isfile(reduced_file_name):
        _reduced_features = read_features(i, src_path, reduced_file_name).to("cpu")
    else:
        if os.path.isfile(file_name):
            _features = read_features(i, src_path, file_name).to("cpu")
        else:
            _features = retrieve_features(test_dataset_images[i].unsqueeze(0).to(device), config["layer_name"], config["sliding_window_size"], config["sliding_window_step"], config["feature_depth"], model).to("cpu")
            save_features(_features, file_name)
        features_transformed = reducer.transform(_features.view(_features.size(0) * _features.size(1), -1))
        _reduced_features = torch.tensor(features_transformed)
        save_features(_reduced_features, reduced_file_name)
    test_reduced_features.append(_reduced_features)
reduced_test_features_tensor = torch.concat(test_reduced_features)
torch.cuda.empty_cache()

In [159]:
reduced_test_features_tensor.view(test_size, _features.size(0), _features.size(1), -1).shape

torch.Size([10, 47, 47, 64])

In [ ]:
step = 48
predicted_masks = []
test_pictures = reduced_test_features_tensor.view(test_size, _features.size(0), _features.size(1), -1)
for t_i in range(test_size):
    test_picture = test_pictures[t_i].to(device)
    predicted_mask = torch.zeros((_features.size(0), _features.size(1)))
    for i in range(0, _features.size(0), step):
        i_upper_limit = min(i+step, _features.size(1))
        predicted_classes = knn.predict(test_picture[i : i_upper_limit].view(-1, test_pictures.size(3)))
        torch.cuda.empty_cache()
        predicted_mask[i : i_upper_limit] = predicted_classes.view(-1, _features.size(1)).to("cpu")
        print(i)
    predicted_masks.append(predicted_mask)

In [ ]:
file_name = os.path.join(src_path, "features", 
                         "predicted_images_{train_size}_{test_size}_{resize_size}.lz4".format(
                               train_size = train_size, test_size = test_size, resize_size = config["resize_size"]
                         ))
with lz4.frame.open(file_name, mode="wb") as f:
      pickle.dump(predicted_masks, f)

c:\ProgramData\anaconda3\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [167]:
predicted_and_ground_truth = []
for i in range(test_size):
    original_size = test_dataset[i]['label'].size
    to_original_size = T.Resize(size=(original_size[1], original_size[0]), interpolation=T.InterpolationMode.NEAREST_EXACT)
    predicted_mask_of_original_size = to_original_size(predicted_masks[i].unsqueeze(0))
    # predicted_masks_of_original_size.append(predicted_mask_of_original_size)
    ground_truth = mask_to_tensor(test_dataset[i]['label'])
    predicted_and_ground_truth.append((predicted_mask_of_original_size.to(torch.int64), ground_truth.to(torch.int64)))

In [ ]:
import torchmetrics
from torchmetrics.segmentation import MeanIoU

In [168]:
result_metrics = torch.zeros((test_size, 6))
for i in range(test_size):
    predicted, ground_truth = predicted_and_ground_truth[i]
    acc = torchmetrics.functional.accuracy(predicted.to(device), ground_truth.to(device), task="multiclass", num_classes=5).to("cpu")
    meanIoU = MeanIoU(num_classes=5, input_format='index', include_background=True, per_class=True).to(device)
    meanIoU_result = meanIoU(predicted.to(device), ground_truth.to(device)).to("cpu")
    result_metrics[i][0] = acc
    result_metrics[i][1:6] = meanIoU_result

In [169]:
mean_results = result_metrics.mean(dim=0)
print("Average accuracy: ", mean_results[0].item())
print("Average meanIoU_result: ", mean_results[1:6])

Average accuracy:  0.826777458190918
Average meanIoU_result:  tensor([0.8187, 0.3792, 0.0668, 0.0000, 0.0050])
